In [10]:
import pandas as pd
import numpy as np
import random
import math

In [11]:
fifa21 = pd.read_csv('FIFA21_official_data.csv')

In [12]:
fifa21.columns

Index(['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Best Position', 'Best Overall Rating', 'Release Clause',
       'DefensiveAwareness'],
      dtype='object')

In [13]:
relavent_fields = ['ID', 'Name', 'Overall', 'Club', 'Jersey Number',
                   'Finishing', 'ShortPassing', 'LongPassing',
                   'Best Position', 'Best Overall Rating']

In [14]:
players = fifa21[relavent_fields]

In [15]:
players = players[players['Overall'] >= 60]

In [16]:
barcelona_players = players[players['Club'] == 'FC Barcelona']
city_players = players[players['Club'] == 'Manchester City']

## Barcelona starters

In [17]:
barcelona_players = barcelona_players.sort_values('Jersey Number')

In [18]:
barcelona_starters_id = [192448, 152729, 189511, 194765, 180206, 158023, 231443, 220440, 189332, 199564, 228702]

In [19]:
barcelona_starters = barcelona_players[barcelona_players['ID'].isin(barcelona_starters_id)]
barcelona_starters.to_csv('barcelona_starters.csv')

## City starters

In [20]:
city_players = city_players.sort_values('Jersey Number')

In [21]:
city_starters_id = [188377, 202652, 153079, 212218, 192985, 218667, 135507, 210257, 204884, 203574, 231866]
city_starters = city_players[city_players['ID'].isin(city_starters_id)]
city_starters.to_csv('city_starters.csv')

In [22]:
city_starters

,ID,Name,Overall,Club,Jersey Number,Finishing,ShortPassing,LongPassing,Best Position,Best Overall Rating
53,188377,K. Walker,85,Manchester City,2.0,51.0,78.0,76.0,RB,85.0
3415,203574,J. Stones,80,Manchester City,5.0,29.0,80.0,75.0,CB,82.0
208,202652,R. Sterling,88,Manchester City,7.0,85.0,84.0,69.0,LW,88.0
182,153079,S. Agüero,89,Manchester City,10.0,94.0,83.0,64.0,ST,89.0
1042,212218,A. Laporte,87,Manchester City,14.0,45.0,82.0,80.0,CB,88.0
257,231866,Rodri,85,Manchester City,16.0,64.0,85.0,82.0,CDM,85.0
1,192985,K. De Bruyne,91,Manchester City,17.0,82.0,94.0,93.0,CAM,91.0
167,218667,Bernardo Silva,87,Manchester City,20.0,79.0,86.0,76.0,RW,87.0
581,204884,B. Mendy,81,Manchester City,22.0,39.0,80.0,69.0,LWB,82.0
45,135507,Fernandinho,84,Manchester City,25.0,66.0,86.0,81.0,CB,84.0


## Simulating gameplay

In [23]:
random.seed(34)

In [24]:
barcelona_starters['scoring likelihood'] = barcelona_starters.apply(lambda row: row.Finishing, axis=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
barcelona_starters

,ID,Name,Overall,Club,Jersey Number,Finishing,ShortPassing,LongPassing,Best Position,Best Overall Rating,scoring likelihood
13611,192448,M. ter Stegen,90,FC Barcelona,1.0,14.0,61.0,63.0,GK,90.0,14.0
962,152729,Piqué,86,FC Barcelona,3.0,65.0,81.0,81.0,CB,86.0,65.0
692,189511,Sergio Busquets,87,FC Barcelona,5.0,67.0,89.0,83.0,CDM,87.0,67.0
3,194765,A. Griezmann,87,FC Barcelona,7.0,88.0,84.0,82.0,ST,87.0,88.0
59,180206,M. Pjanić,85,FC Barcelona,8.0,56.0,90.0,87.0,CM,85.0,56.0
18,158023,L. Messi,93,FC Barcelona,10.0,95.0,91.0,91.0,RW,93.0,95.0
978,231443,O. Dembélé,83,FC Barcelona,11.0,75.0,78.0,76.0,LM,85.0,75.0
1781,220440,C. Lenglet,85,FC Barcelona,15.0,39.0,78.0,74.0,CB,86.0,39.0
21,189332,Jordi Alba,86,FC Barcelona,18.0,73.0,84.0,79.0,LB,86.0,73.0
254,199564,Sergi Roberto,83,FC Barcelona,20.0,63.0,84.0,78.0,RB,83.0,63.0


In [18]:
#likelihood of each barcelona player scoring

gk = [1]
defense = [3,15,18,20]
mid = [5,8,21]
forwards = [7,10,11]
weights = list(barcelona_starters['Finishing'])

def adjust_likelihood(jersey_num, initial_likelihood):
    if jersey_num in gk:
        return initial_likelihood/100
    if jersey_num in defense:
        return initial_likelihood
    if jersey_num in mid:
        return initial_likelihood * 2
    if jersey_num in forwards:
        return initial_likelihood * 4 

In [19]:
barcelona_starters['scoring likelihood'] = barcelona_starters.apply(lambda row: adjust_likelihood(row['Jersey Number'], row['scoring likelihood']), axis = 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
barcelona_starters

NameError: name 'barcelona_starters' is not defined

In [21]:
#likelihood of each city player scoring

city_starters['scoring likelihood'] = city_starters.apply(lambda row: row.Finishing, axis=1)

gk = [31]
defense = [2, 5, 14, 22]
mid = [17, 25, 16]
forwards = [7, 10, 20]

weights = list(city_starters['Finishing'])

city_starters['scoring likelihood'] = city_starters.apply(
                                                    lambda row: adjust_likelihood(row['Jersey Number'], row['scoring likelihood']), 
                                                    axis = 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
barcelona_starters
city_starters

,ID,Name,Overall,Club,Jersey Number,Finishing,ShortPassing,LongPassing,Best Position,Best Overall Rating,scoring likelihood
53,188377,K. Walker,85,Manchester City,2.0,51.0,78.0,76.0,RB,85.0,51.00
3415,203574,J. Stones,80,Manchester City,5.0,29.0,80.0,75.0,CB,82.0,29.00
208,202652,R. Sterling,88,Manchester City,7.0,85.0,84.0,69.0,LW,88.0,340.00
182,153079,S. Agüero,89,Manchester City,10.0,94.0,83.0,64.0,ST,89.0,376.00
1042,212218,A. Laporte,87,Manchester City,14.0,45.0,82.0,80.0,CB,88.0,45.00
257,231866,Rodri,85,Manchester City,16.0,64.0,85.0,82.0,CDM,85.0,128.00
1,192985,K. De Bruyne,91,Manchester City,17.0,82.0,94.0,93.0,CAM,91.0,164.00
167,218667,Bernardo Silva,87,Manchester City,20.0,79.0,86.0,76.0,RW,87.0,316.00
581,204884,B. Mendy,81,Manchester City,22.0,39.0,80.0,69.0,LWB,82.0,39.00
45,135507,Fernandinho,84,Manchester City,25.0,66.0,86.0,81.0,CB,84.0,132.00


In [23]:
population = list(city_starters['Name'])
weights = list(city_starters['scoring likelihood'])
goals = dict()
for i in range(5000):
    curr_choice = random.choices([x for x in range(0,11)], weights)[0]
    if population[curr_choice] in goals.keys():
        goals[population[curr_choice]]+=1
    else:
        goals[population[curr_choice]] = 1
goals

{' Rodri': 403,
 ' Bernardo Silva': 935,
 ' B. Mendy': 125,
 ' S. Agüero': 1177,
 ' Fernandinho': 402,
 ' R. Sterling': 1033,
 ' A. Laporte': 153,
 ' K. De Bruyne': 514,
 ' J. Stones': 99,
 ' K. Walker': 159}

In [29]:
def game():
    population = [0,1]
    weights = [29/30, 1/30]
    
    team1_score = 0
    team2_score = 0
    team1_scorers = []
    team2_scorers = []

    for i in range(90):
        team_1 = random.choices(population, weights)[0]
        team_2 = random.choices(population, weights)[0]
        
        if team_1 == 1:   #team1 scores
            scorer_population = list(barcelona_starters['Name'])
            scorer_weights = list(barcelona_starters['scoring likelihood'])
            curr_choice = random.choices([x for x in range(0,11)], scorer_weights)[0]
            curr_scorer = scorer_population[curr_choice]
            team1_scorers.append(curr_scorer)
            print("Goal Barcelona {}' - {}".format(i, curr_scorer))
            team1_score+=1
            
        if team_2 == 1:    #team2 scores
            scorer_population = list(city_starters['Name'])
            scorer_weights = list(barcelona_starters['scoring likelihood'])
            curr_choice = random.choices([x for x in range(0,11)], scorer_weights)[0]
            curr_scorer = scorer_population[curr_choice]
            team2_scorers.append(curr_scorer)
            print("Goal Manchester City {}' - {}' ".format(i, curr_scorer))
            team2_score+=1
#     minute = 0
#     team1_score = math.floor(random.random() * 5)
#     team2_score = math.floor(random.random() * 5)
#     while(minute<=90):
        
    return 'Final Score: {} - {}'.format(team1_score, team2_score)

In [40]:
game()

Goal Manchester City 10' -  S. Agüero' 
Goal Manchester City 12' -  K. De Bruyne' 
Goal Manchester City 29' -  S. Agüero' 
Goal Manchester City 32' -  Rodri' 
Goal Barcelona 36' -  F. de Jong
Goal Barcelona 51' -  F. de Jong


'Final Score: 2 - 4'

In [56]:
barcelona_starters

,ID,Name,Overall,Club,Jersey Number,Finishing,ShortPassing,LongPassing,Best Position,Best Overall Rating,scoring likelihood
13611,192448,M. ter Stegen,90,FC Barcelona,1.0,14.0,61.0,63.0,GK,90.0,0.14
962,152729,Piqué,86,FC Barcelona,3.0,65.0,81.0,81.0,CB,86.0,65.00
692,189511,Sergio Busquets,87,FC Barcelona,5.0,67.0,89.0,83.0,CDM,87.0,134.00
3,194765,A. Griezmann,87,FC Barcelona,7.0,88.0,84.0,82.0,ST,87.0,352.00
59,180206,M. Pjanić,85,FC Barcelona,8.0,56.0,90.0,87.0,CM,85.0,112.00
18,158023,L. Messi,93,FC Barcelona,10.0,95.0,91.0,91.0,RW,93.0,380.00
978,231443,O. Dembélé,83,FC Barcelona,11.0,75.0,78.0,76.0,LM,85.0,300.00
1781,220440,C. Lenglet,85,FC Barcelona,15.0,39.0,78.0,74.0,CB,86.0,39.00
21,189332,Jordi Alba,86,FC Barcelona,18.0,73.0,84.0,79.0,LB,86.0,73.00
254,199564,Sergi Roberto,83,FC Barcelona,20.0,63.0,84.0,78.0,RB,83.0,63.00


In [145]:
population = [0,1]
weights = [1, 0]

In [147]:
res = []
for i in range(90):
    res.append(random.choices(population, weights)[0])
    
res.count(1)

0

In [2]:
import pygame

pygame 2.0.0.dev4 (SDL 2.0.10, python 3.7.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
pygame.font.SysFont?

Signature: pygame.font.SysFont(name, size, bold=False, italic=False, constructor=None)
Docstring:
pygame.font.SysFont(name, size, bold=False, italic=False, constructor=None) -> Font
create a pygame Font from system font resources

This will search the system fonts for the given font
name. You can also enable bold or italic styles, and
the appropriate system font will be selected if available.

This will always return a valid Font object, and will
fallback on the builtin pygame font if the given font
is not found.

Name can also be a comma separated list of names, in
which case set of names will be searched in order. Pygame
uses a small set of common font aliases, if the specific
font you ask for is not available, a reasonable alternative
may be used.

if optional contructor is provided, it must be a function with
signature constructor(fontpath, size, bold, italic) which returns
a Font instance. If None, a pygame.font.Font object is created.
File:      /usr/local/lib/python3.7/site-pack